In [68]:
import pyspiel
%reload_ext autoreload
%autoreload 2
from fcpa_agent import *
from open_spiel.python.bots import human
import numpy as np

In [69]:
agent_id = 0
human_id = 1-agent_id
agent = get_agent_for_tournament(agent_id)
human_player = human.HumanBot()
if agent_id == 1:
  agents.clear()
  agents.append(agent)
  agents.append(human_player)
else:
  agents.append(human_player)
  agents.append(agent)


<class 'open_spiel.python.algorithms.deep_cfr_tf2.PolicyNetwork'>
loaded model


2022-05-18 14:26:51.112966: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [70]:
fcpa_game_string = (
        "universal_poker(betting=nolimit,numPlayers=2,numRounds=4,blind=150 100,"
        "firstPlayer=2 1 1 1,numSuits=4,numRanks=13,numHoleCards=2,numBoardCards=0 3 1 1,"
        "stack=20000 20000,bettingAbstraction=fcpa)")
game = pyspiel.load_game(fcpa_game_string)

In [75]:
rng = np.random.RandomState()
state = game.new_initial_state()
while not state.is_terminal():
  # The state can be three different types: chance node,
  # simultaneous node, or decision node
  current_player = state.current_player()
  if current_player == human_id:
    print("####STATE####")
    print(state.information_state_string(human_id))
  if state.is_chance_node():
    # Chance node: sample an outcome
    outcomes = state.chance_outcomes()
    num_actions = len(outcomes)
    print("Chance node with " + str(num_actions) + " outcomes")
    action_list, prob_list = zip(*outcomes)
    action = rng.choice(action_list, p=prob_list)
    print("Sampled outcome: ",
          state.action_to_string(state.current_player(), action))
    state.apply_action(action)
  else:
    # Decision node: sample action for the single current player
    legal_actions = state.legal_actions()
    for action in legal_actions:
      print("Legal action: {} ({})".format(
          state.action_to_string(current_player, action), action))
    action = agents[current_player].step(state)
    action_string = state.action_to_string(current_player, action)
    print("Player ", current_player, ", chose action: ",
          action_string)
    state.apply_action(action)

# Game is now done. Print utilities for each player
print(state)
returns = state.returns()
for pid in range(game.num_players()):
  print("Utility for player {} is {}".format(pid, returns[pid]))

Chance node with 52 outcomes
Sampled outcome:  player=-1 move=Deal(29)
Chance node with 51 outcomes
Sampled outcome:  player=-1 move=Deal(48)
Chance node with 50 outcomes
Sampled outcome:  player=-1 move=Deal(22)
Chance node with 49 outcomes
Sampled outcome:  player=-1 move=Deal(15)
####STATE####
[Round 0][Player: 1][Pot: 300][Money: 19850 19900][Private: 7h5s][Public: ][Sequences: ]
Legal action: player=1 move=Fold (0)
Legal action: player=1 move=Call (1)
Legal action: player=1 move=Bet (2)
Legal action: player=1 move=AllIn (3)
Player  1 , chose action:  player=1 move=Call
Legal action: player=0 move=Call (1)
Legal action: player=0 move=Bet (2)
Legal action: player=0 move=AllIn (3)
{1: 0.001765426, 2: 0.05230988, 3: 0.94592476}
[1, 2, 3]
2
Player  0 , chose action:  player=0 move=Bet
####STATE####
[Round 0][Player: 1][Pot: 900][Money: 19550 19850][Private: 7h5s][Public: ][Sequences: cr450]
Legal action: player=1 move=Fold (0)
Legal action: player=1 move=Call (1)
Legal action: player=1